In [1]:
from DOS import *
import time

In [2]:
r = 0.05  #interest rate
dividend = 0.1  #divident rate
seed_train = 1 # seed for training data
seed_test = 2 # seed for reproducibility
learning_rate = 0.001 #learning rate
k = 100.0  #strike price
test_paths = 100000 # number of paths for testing

In [3]:
# Parameters
d = 5   #dimension of GBM
mu = (r - dividend) * np.ones(shape = (d, )) # drift
sigma = 0.2 * np.ones(shape = (d, ))
S0 = 100.0 * np.ones(shape = (d, )) #initial price
# we use lessp paths due to limited computational resources
base_steps = 1500 # number of training steps regardless of the dimension
total_paths = batch_size * (base_steps + d) #total number of paths, base_steps+d is the training steps, where each step we need 8192 paths
number_of_training_steps = int(total_paths / batch_size) #number of training steps

#Simulating GBM paths
X = GBM(d, mu, sigma, S0, T, dt, total_paths, seed=seed_train) #simulating GBM paths
X = X.reshape(base_steps+d, batch_size, d, 10) #reshaping to (3000+d, 8192, d, 10) to simulate 3000+d many paths of 8192 samples of 5 dimensional GBM

In [4]:
# training the model for each stopping time
start_time = time.time()
for i in range(N-1, 0, -1):
    model = create_model(d).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)
    train(X, r, k, dt, model, i, optimizer, number_of_training_steps, batch_size)
    # store the model and the stopping decision function and stopping time function
    F[i] = model
    f[i] = lambda x, i=i: fi(x, i, F) #  store the stopping decision function
    l[i] = lambda x, i=i: li(x, i, f, l) # store the stopping time function
end_time = time.time()

the model for 8th stopping time has been trained
the model for 7th stopping time has been trained
the model for 6th stopping time has been trained
the model for 5th stopping time has been trained
the model for 4th stopping time has been trained
the model for 3th stopping time has been trained
the model for 2th stopping time has been trained
the model for 1th stopping time has been trained


In [5]:
# test the model on another set of paths
X = GBM(d, mu, sigma, S0, T, dt, test_paths, seed = seed_test)
g_val = g(X, r, k, dt) # g values at stopping times
X = torch.from_numpy(X).float().to(device) # convert X to a tensor
Z = g_val[range(test_paths), l[1](X)] # g values at stopping times, l[1] 
price = 1 / test_paths * np.sum(Z) # monte carlo estimate of the price
print(f"Estimated price of the {d}D Bermudan Max Call Option: {price:.2f}")
print(f"Time for training the model with {d} dimensions: {end_time - start_time:.2f} seconds")

Estimated price of the 5D Bermudan Max Call Option: 26.03
Time for training the model with 5 dimensions: 120.01 seconds
